# Vector Database



Um Vector Database (ou banco de dados vetorial) é um tipo de banco de dados especializado no armazenamento, indexação e busca eficiente de vetores de alta dimensão — representações numéricas densas, geralmente extraídas de dados como textos, imagens, vídeos ou áudio.

Esses vetores são a base de aplicações de IA modernas, como:

Busca semântica (semantic search)

Recuperação aumentada por geração (RAG)

Recomendação de conteúdo

Detecção de similaridade

🧠 Por que vetores?
Modelos de machine learning (como BERT, CLIP, OpenAI embeddings etc.) convertem entradas complexas (como textos) em vetores numéricos. Cada vetor representa o "significado" daquele dado em um espaço matemático. Com isso, é possível medir semelhança semântica usando métricas como:

Distância Euclidiana

Cosseno

Inner product

🌲 Pinecone – O que é?

Pinecone é uma Vector Database as a Service, gerenciada, escalável e de alta performance. Ela permite que você:

Armazene bilhões de vetores

Faça buscas vetoriais rápidas

Integre facilmente com LLMs (ex: OpenAI, Cohere)

Use com frameworks como LangChain, LlamaIndex, Haystack

🛠️ Como funciona a arquitetura básica?
Criação de vetores: Um modelo de IA transforma dados (como textos) em vetores.

Indexação: Esses vetores são armazenados em Pinecone.

Consulta: Você envia um vetor de consulta, e Pinecone retorna os vetores mais semelhantes.

Integração com contexto: Em sistemas RAG, os resultados recuperados podem ser usados como contexto para LLMs.




# 🔍 RAG com Pinecone + MongoDB + OpenAI

Este notebook demonstra como construir um sistema de **Pergunta e Resposta com Busca Semântica** (RAG - Retrieval-Augmented Generation), integrando:
- Vetores semânticos com OpenAI
- Armazenamento vetorial com Pinecone
- Persistência dos documentos em MongoDB
- Geração de resposta com contexto usando ChatGPT (GPT-3.5-turbo)

Ideal para casos de uso como:
- FAQ inteligentes
- Chatbots com memória contextual
- Sistemas de suporte baseados em conhecimento

Para obter suas chaves de API do **Pinecone** e **OpenAI**, siga os passos abaixo:

---

### 🔑 **1. Como obter a API Key do Pinecone**

1. Acesse: [https://www.pinecone.io](https://www.pinecone.io)
2. Clique em **“Start for free”** e crie uma conta (ou faça login).
3. No Dashboard:

   * Vá para **API Keys** (menu lateral esquerdo).
   * Copie a **API Key** (ex: `a1b2c3d4...`) e o **Environment** (ex: `gcp-starter`, `us-west1-gcp`).

⚠️ Guarde essas informações, você usará assim:

```python
pinecone_api_key = "SUA_PINECONE_API_KEY"
pinecone_env = "us-west1-gcp"
```

---

### 🔐 **2. Como obter a OpenAI API Key**

1. Acesse: [https://platform.openai.com/account/api-keys](https://platform.openai.com/account/api-keys)
2. Faça login com sua conta da OpenAI.
3. Clique em **“Create new secret key”**.
4. Copie e salve a chave (você só verá uma vez!).

Exemplo:

```python
openai_api_key = "sk-..."
```

---

### 💡 Dica de segurança:

* Nunca compartilhe suas chaves publicamente.
* Use variáveis de ambiente (como `os.getenv`) ou arquivos `.env` para maior segurança em produção.

Se quiser, posso mostrar como armazenar essas chaves com segurança em um projeto Colab ou Python local. Deseja isso também?


In [ ]:
# Instalar dependências
!pip install openai pinecone-client pymongo --quiet

In [ ]:
# Configurações (edite com suas chaves)
openai_api_key = "SUA_OPENAI_API_KEY"
pinecone_api_key = "SUA_PINECONE_API_KEY"
pinecone_env = "us-west1-gcp"
pinecone_index_name = "rag-index"

mongodb_uri = "mongodb://localhost:27017"
db_name = "ragdb"
collection_name = "faq"

In [ ]:
# Inicializações
import openai
import pinecone
from pymongo import MongoClient
import uuid

openai.api_key = openai_api_key
pinecone.init(api_key=pinecone_api_key, environment=pinecone_env)

# Pinecone
if pinecone_index_name not in pinecone.list_indexes():
    pinecone.create_index(pinecone_index_name, dimension=1536)
index = pinecone.Index(pinecone_index_name)

# MongoDB
mongo = MongoClient(mongodb_uri)
collection = mongo[db_name][collection_name]

def gerar_embedding(texto):
    response = openai.Embedding.create(input=[texto], model="text-embedding-ada-002")
    return response["data"][0]["embedding"]

In [ ]:
# Inserir base de conhecimento
faq_data = [
    {"pergunta": "Como redefinir minha senha?", "resposta": "Acesse configurações e clique em 'Redefinir Senha'."},
    {"pergunta": "Qual é o horário de atendimento?", "resposta": "Das 8h às 18h, de segunda a sexta."},
    {"pergunta": "Onde posso baixar minha fatura?", "resposta": "Na área do cliente, seção 'Faturas'."}
]

for item in faq_data:
    item_id = str(uuid.uuid4())
    vetor = gerar_embedding(item["pergunta"])
    item["_id"] = item_id
    collection.insert_one(item)
    index.upsert([(item_id, vetor)])

In [ ]:
# Função de busca com recuperação semântica
def buscar_contexto(pergunta, top_k=1):
    embedding = gerar_embedding(pergunta)
    results = index.query(vector=embedding, top_k=top_k)
    contextos = []
    for match in results["matches"]:
        doc = collection.find_one({"_id": match["id"]})
        if doc:
            contextos.append(doc["resposta"])
    return "\n".join(contextos)

In [ ]:
# Função de resposta com contexto RAG
def responder(pergunta):
    contexto = buscar_contexto(pergunta)
    prompt = f"""
Contexto:
{contexto}

Pergunta:
{pergunta}

Responda com base apenas no contexto acima.
"""
    resposta = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.3
    )
    return resposta["choices"][0]["message"]["content"]

In [ ]:
# Teste interativo
while True:
    pergunta = input("Você: ")
    if pergunta.lower() in ["sair", "exit", "quit"]:
        break
    resposta = responder(pergunta)
    print("Bot:", resposta)